In [ ]:
%pip install langchain langchain-openai langchain_chroma langchain_community langchainhub beautifulsoup4


In [ ]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AIKEY"] = getpass.getpass()

token = os.getenv("AIKEY_SECOND")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, base_url=endpoint, openai_api_key=token)


In [ ]:
import bs4
#from bs4 import BeautifulSoup
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lt.wikipedia.org/wiki/Vilnius/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            #class_=("post-content", "post-title", "post-header")
            id="bodyContent"
        )
    ),
)

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embedding = OpenAIEmbeddings(
    base_url="https://models.inference.ai.azure.com",
    model="text-embedding-3-large",
    openai_api_key=openai_api_key
)

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

In [63]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

print(prompt)

C:\Users\Zilvinas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [64]:
def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [65]:
rag_chain.invoke("What is Convolutional Neural Networks?")

[Document(id='8731b045-04ce-45ae-8dbf-912762e48426', metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'}, page_content='Convolutional Neural Network#\nConvolutional neural networks, short for “CNN”, is a type of feed-forward artificial neural networks, in which the connectivity pattern between its neurons is inspired by the organization of the visual cortex system. The primary visual cortex (V1) does edge detection out of the raw visual input from the retina. The secondary visual cortex (V2), also called prestriate cortex, receives the edge features from V1 and extracts simple visual properties such as orientation, spatial frequency, and color. The visual area V4 handles more complicated object attributes. All the processed visual features flow into the final logic unit, inferior temporal gyrus (IT), for object recognition. The shortcut between V1 and V4 inspires a special type of CNN with connections between non-adjacent layers: Residual Net (He, et al. 2016

'Convolutional Neural Networks (CNNs) are a type of feed-forward artificial neural network inspired by the organization of the visual cortex. They process visual information through layers that detect edges, extract features, and recognize objects, often incorporating special connections like residual blocks. CNNs are widely used for image recognition and related tasks.'